In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('inventory.db')

In [3]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables

,name
0,begin_inventory
1,end_inventory
2,purchases
3,purchase_prices
4,sales
5,vendor_invoice


In [45]:
vendor_summary_table= pd.read_sql_query("""
    WITH vendor_summary AS
        (SELECT 
            VendorNumber,
            PODate,
            SUM(Freight) as TotalFreight
        FROM vendor_invoice
        GROUP BY VendorNumber, PODate),

    purchase_summary AS
        (SELECT 
            p.VendorNumber,
            p.VendorName,
            p.PONumber,
            p.PurchasePrice,
            p.PODate,
            pp.Price AS ActualPrice,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS TotalPurchaseDollars,
            p.Brand,
            p.Description
        FROM purchases p
        JOIN purchase_prices pp
        ON p.Brand = pp.Brand
        GROUP BY p.Description, p.Brand, p.VendorNumber, p.VendorName, p.PurchasePrice, pp.Price, pp.Volume, p.PODate),
        
    sales_summary AS
        (SELECT 
            VendorNo,
            Brand,
            Description,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesPrice) AS TotalSalesPrice,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY Brand, Description, VendorNo)

SELECT 
    ps.VendorNumber,
    ps.VendorName,
    ps.PODate AS DateOfPurchase,
    STRFTIME('%m', ps.PODate) AS MonthNumber,
    CASE STRFTIME('%m', ps.PODate)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS MonthOfPurchase,
    STRFTIME('%Y', ps.PODate) AS YearOfPurchase,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ps.Brand,
    ps.PONumber,
    ps.Description,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    vs.TotalFreight
FROM purchase_summary ps
LEFT JOIN sales_summary ss
    ON ps.VendorNumber = ss.VendorNo
        AND ps.Brand = ss.Brand
LEFT JOIN vendor_summary vs
    ON ps.VendorNumber = vs.VendorNumber
         AND ps.PODate = vs.PODate
""", conn)

In [46]:
vendor_summary_table.isnull().sum()

VendorNumber                 0
VendorName                   0
DateOfPurchase               0
MonthNumber                  0
MonthOfPurchase              0
YearOfPurchase               0
PurchasePrice                0
ActualPrice                  0
TotalPurchaseQuantity        0
TotalPurchaseDollars         0
Brand                        0
PONumber                     0
Description                  0
TotalSalesQuantity         182
TotalSalesDollars          182
TotalSalesPrice            182
TotalExciseTax             182
TotalFreight             46729
dtype: int64

In [47]:
vendor_summary_table.fillna({
    'TotalSalesQuantity': 0,
    'TotalSalesDollars': 0,
    'TotalSalesPrice': 0,
    'TotalExciseTax': 0,
    'TotalFreight': 0
}, inplace=True)

In [48]:
vendor_summary_table.isnull().sum()

VendorNumber             0
VendorName               0
DateOfPurchase           0
MonthNumber              0
MonthOfPurchase          0
YearOfPurchase           0
PurchasePrice            0
ActualPrice              0
TotalPurchaseQuantity    0
TotalPurchaseDollars     0
Brand                    0
PONumber                 0
Description              0
TotalSalesQuantity       0
TotalSalesDollars        0
TotalSalesPrice          0
TotalExciseTax           0
TotalFreight             0
dtype: int64

In [49]:
vendor_summary_table.columns

Index(['VendorNumber', 'VendorName', 'DateOfPurchase', 'MonthNumber',
       'MonthOfPurchase', 'YearOfPurchase', 'PurchasePrice', 'ActualPrice',
       'TotalPurchaseQuantity', 'TotalPurchaseDollars', 'Brand', 'PONumber',
       'Description', 'TotalSalesQuantity', 'TotalSalesDollars',
       'TotalSalesPrice', 'TotalExciseTax', 'TotalFreight'],
      dtype='object')

In [50]:
vendor_summary_table['VendorName'] = vendor_summary_table['VendorName'].str.strip()
vendor_summary_table['Description'] = vendor_summary_table['Description'].str.strip()

In [51]:
vendor_summary_table['GrossProfit'] = vendor_summary_table['TotalSalesDollars'] - vendor_summary_table['TotalPurchaseQuantity']

In [52]:
vendor_summary_table['ProfitMargin'] = vendor_summary_table['GrossProfit'] / vendor_summary_table['TotalSalesDollars'] * 100

In [53]:
vendor_summary_table['SalesToPurchaseRatio'] = vendor_summary_table['TotalSalesDollars'] / vendor_summary_table['TotalPurchaseQuantity']

In [54]:
vendor_summary_table['TotalStockTurnOver'] = vendor_summary_table['TotalSalesQuantity'] / vendor_summary_table['TotalSalesDollars'] * 100

In [55]:
vendor_summary_table

,VendorNumber,VendorName,DateOfPurchase,MonthNumber,MonthOfPurchase,YearOfPurchase,PurchasePrice,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars,...,Description,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,TotalFreight,GrossProfit,ProfitMargin,SalesToPurchaseRatio,TotalStockTurnOver
0,12546,JIM BEAM BRANDS COMPANY,2023-12-22,12,December,2023,26.92,36.99,6,161.52,...,(RI) 1,591.0,21519.09,19362.68,466.43,3506.08,21513.09,99.972118,3586.515000,2.746399
1,12546,JIM BEAM BRANDS COMPANY,2023-12-29,12,December,2023,26.92,36.99,6,161.52,...,(RI) 1,591.0,21519.09,19362.68,466.43,2910.40,21513.09,99.972118,3586.515000,2.746399
2,12546,JIM BEAM BRANDS COMPANY,2024-01-04,01,January,2024,26.92,36.99,18,484.56,...,(RI) 1,591.0,21519.09,19362.68,466.43,2027.87,21501.09,99.916353,1195.505000,2.746399
3,12546,JIM BEAM BRANDS COMPANY,2024-01-12,01,January,2024,26.92,36.99,6,161.52,...,(RI) 1,591.0,21519.09,19362.68,466.43,1602.01,21513.09,99.972118,3586.515000,2.746399
4,12546,JIM BEAM BRANDS COMPANY,2024-01-17,01,January,2024,26.92,36.99,27,726.84,...,(RI) 1,591.0,21519.09,19362.68,466.43,1856.19,21492.09,99.874530,797.003333,2.746399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228881,9165,ULTRA BEVERAGE COMPANY LLP,2024-05-02,05,May,2024,8.78,12.99,24,210.72,...,g Sake,158.0,819.92,239.10,6.99,1118.87,795.92,97.072885,34.163333,19.270173
228882,9165,ULTRA BEVERAGE COMPANY LLP,2024-05-13,05,May,2024,8.78,12.99,2,17.56,...,g Sake,158.0,819.92,239.10,6.99,1257.30,817.92,99.756074,409.960000,19.270173
228883,9165,ULTRA BEVERAGE COMPANY LLP,2024-08-08,08,August,2024,8.78,12.99,48,421.44,...,g Sake,158.0,819.92,239.10,6.99,2427.83,771.92,94.145770,17.081667,19.270173
228884,9165,ULTRA BEVERAGE COMPANY LLP,2024-08-31,08,August,2024,8.78,12.99,84,737.52,...,g Sake,158.0,819.92,239.10,6.99,1368.83,735.92,89.755098,9.760952,19.270173


In [56]:
vendor_summary_table.to_csv("data/Vendor_performance_dataset/vendor_performance_summary.csv", index=False)

In [60]:
delivery_timing_table = pd.read_sql_query("""
    SELECT 
        p.VendorNumber,
        p.PONumber,
        p.PODate AS DateOfPurchase,
        STRFTIME('%m', p.PODate) AS MonthNumber,
        CASE STRFTIME('%m', p.PODate)
            WHEN '01' THEN 'January'
            WHEN '02' THEN 'February'
            WHEN '03' THEN 'March'
            WHEN '04' THEN 'April'
            WHEN '05' THEN 'May'
            WHEN '06' THEN 'June'
            WHEN '07' THEN 'July'
            WHEN '08' THEN 'August'
            WHEN '09' THEN 'September'
            WHEN '10' THEN 'October'
            WHEN '11' THEN 'November'
            WHEN '12' THEN 'December'
        END AS MonthOfPurchase,
        STRFTIME('%Y', p.PODate) AS YearOfPurchase,
        p.ReceivingDate,
        vi.InvoiceDate,
        vi.PayDate,
        p.Brand,
        p.Description,
        p.Store,

        -- Derived metrics
        JULIANDAY(p.ReceivingDate) - JULIANDAY(p.PODate) AS DeliveryTimeInDays,
        JULIANDAY(vi.InvoiceDate) - JULIANDAY(p.ReceivingDate) AS InvoiceProcessingTimeInDays,
        JULIANDAY(vi.PayDate) - JULIANDAY(vi.InvoiceDate) AS PaymentProcessingTimeInDays,
        JULIANDAY(vi.PayDate) - JULIANDAY(p.PODate) AS TotalCycleTimeInDays

    FROM purchases p
    JOIN vendor_invoice vi
        ON p.PONumber = vi.PONumber
""", conn)


In [61]:
delivery_timing_table[:10]

,VendorNumber,PONumber,DateOfPurchase,MonthNumber,MonthOfPurchase,YearOfPurchase,ReceivingDate,InvoiceDate,PayDate,Brand,Description,Store,DeliveryTimeInDays,InvoiceProcessingTimeInDays,PaymentProcessingTimeInDays,TotalCycleTimeInDays
0,105,8124,2023-12-21,12,December,2023,2024-01-02,2024-01-04,2024-02-16,8412,Tequila Ocho Plata Fresno,69,12.0,2.0,43.0,57.0
1,4466,8137,2023-12-22,12,December,2023,2024-01-01,2024-01-07,2024-02-21,5255,TGI Fridays Ultimte Mudslide,30,10.0,6.0,45.0,61.0
2,4466,8137,2023-12-22,12,December,2023,2024-01-02,2024-01-07,2024-02-21,5215,TGI Fridays Long Island Iced,34,11.0,5.0,45.0,61.0
3,4466,8137,2023-12-22,12,December,2023,2024-01-01,2024-01-07,2024-02-21,5255,TGI Fridays Ultimte Mudslide,1,10.0,6.0,45.0,61.0
4,388,8169,2023-12-24,12,December,2023,2024-01-02,2024-01-09,2024-02-16,2034,Glendalough Double Barrel,76,9.0,7.0,38.0,54.0
5,480,8106,2023-12-20,12,December,2023,2024-01-02,2024-01-12,2024-02-05,3348,Bombay Sapphire Gin,5,13.0,10.0,24.0,47.0
6,480,8106,2023-12-20,12,December,2023,2024-01-01,2024-01-12,2024-02-05,8358,Bacardi 151 Proof,1,12.0,11.0,24.0,47.0
7,480,8106,2023-12-20,12,December,2023,2024-01-01,2024-01-12,2024-02-05,4903,Bacardi Superior Rum,30,12.0,11.0,24.0,47.0
8,480,8106,2023-12-20,12,December,2023,2024-01-02,2024-01-12,2024-02-05,3782,Grey Goose Le Citron Vodka,34,13.0,10.0,24.0,47.0
9,480,8106,2023-12-20,12,December,2023,2024-01-01,2024-01-12,2024-02-05,4233,Castillo Silver Label Rum,1,12.0,11.0,24.0,47.0


In [62]:
delivery_timing_table.isnull().sum()

VendorNumber                   0
PONumber                       0
DateOfPurchase                 0
MonthNumber                    0
MonthOfPurchase                0
YearOfPurchase                 0
ReceivingDate                  0
InvoiceDate                    0
PayDate                        0
Brand                          0
Description                    0
Store                          0
DeliveryTimeInDays             0
InvoiceProcessingTimeInDays    0
PaymentProcessingTimeInDays    0
TotalCycleTimeInDays           0
dtype: int64

In [63]:
delivery_timing_table.to_csv('data/Vendor_performance_dataset/delivery_timing_summary.csv', index=False)

In [67]:
purchases = pd.read_sql_query("SELECT * FROM purchases WHERE VendorNumber=4466", conn)

In [68]:
purchases[:10]

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
5,67_EANVERNESS_5215,67,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-08,2024-01-19,2024-02-26,9.41,6,56.46,1
6,48_NORFOLK_5255,48,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.35,6,56.10,1
7,74_PAENTMARWY_5215,74,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
8,23_ARBINGTON_5255,23,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.35,6,56.10,1
9,14_BROMWICH_5215,14,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-06,2024-01-19,2024-02-26,9.41,6,56.46,1


In [8]:
purchases.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'VendorNumber',
       'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
       'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification'],
      dtype='object')

In [69]:
# summary = purchases.groupby(['VendorName', 'PODate'])['PurchasePrice'].sum().reset_index().rename(columns={'PurchasePrice':'Total_Purchase_Price'})

vendors_purchase_details_by_podate = purchases.groupby(['VendorName', 'PODate']).agg(Total_Purchase_price = ('PurchasePrice','sum'),Total_quantity = ('Quantity', 'sum'), Total_Dollars=('Dollars','sum')).reset_index().sort_values('PODate')

In [70]:
vendors_purchase_details_by_podate

,VendorName,PODate,Total_Purchase_price,Total_quantity,Total_Dollars
0,AMERICAN VINTAGE BEVERAGE,2023-12-22,28.11,15,140.55
1,AMERICAN VINTAGE BEVERAGE,2023-12-27,281.16,335,3142.33
2,AMERICAN VINTAGE BEVERAGE,2024-01-03,65.45,41,383.35
3,AMERICAN VINTAGE BEVERAGE,2024-01-14,112.20,72,673.20
4,AMERICAN VINTAGE BEVERAGE,2024-01-19,131.20,79,740.21
5,AMERICAN VINTAGE BEVERAGE,2024-01-24,526.30,347,3261.37
6,AMERICAN VINTAGE BEVERAGE,2024-02-05,112.56,72,675.36
7,AMERICAN VINTAGE BEVERAGE,2024-02-12,159.25,117,1096.05
8,AMERICAN VINTAGE BEVERAGE,2024-02-19,206.24,129,1209.27
9,AMERICAN VINTAGE BEVERAGE,2024-02-22,196.83,147,1377.87


In [76]:
vendor_invoice = pd.read_sql_query("SELECT * FROM vendor_invoice WHERE VendorNumber=4466", conn)

In [77]:
vendor_invoice

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None
5,4466,AMERICAN VINTAGE BEVERAGE,2024-02-09,8604,2024-01-24,2024-03-15,347,3261.37,17.61,None
6,4466,AMERICAN VINTAGE BEVERAGE,2024-02-17,8793,2024-02-05,2024-04-02,72,675.36,3.17,None
7,4466,AMERICAN VINTAGE BEVERAGE,2024-03-01,8892,2024-02-12,2024-03-28,117,1096.05,5.15,None
8,4466,AMERICAN VINTAGE BEVERAGE,2024-03-07,8995,2024-02-19,2024-04-02,129,1209.27,5.44,None
9,4466,AMERICAN VINTAGE BEVERAGE,2024-03-12,9033,2024-02-22,2024-04-16,147,1377.87,6.61,None


In [79]:
vendor_invoice[(vendor_invoice['VendorName'].str.contains('AMERICAN VINTAGE')) & (vendor_invoice['PODate'] == '2023-12-27')]

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None


In [80]:
brands_purchase_details_by_podate = purchases.groupby(['Brand', 'PODate']).agg(Total_Purchase_price = ('PurchasePrice','sum'),Total_quantity = ('Quantity', 'sum'), Total_Dollars=('Dollars','sum')).reset_index().sort_values('PODate')

In [81]:
brands_purchase_details_by_podate

,Brand,PODate,Total_Purchase_price,Total_quantity,Total_Dollars
90,5255,2023-12-22,18.70,10,93.50
39,5215,2023-12-22,9.41,5,47.05
40,5215,2023-12-27,103.51,168,1580.88
91,5255,2023-12-27,177.65,167,1561.45
92,5255,2024-01-03,65.45,41,383.35
...,...,...,...,...,...
138,5255,2024-12-11,196.35,106,991.10
37,3140,2024-12-11,123.09,59,660.21
89,5215,2024-12-20,188.20,126,1185.66
38,3140,2024-12-20,111.90,63,704.97


In [61]:
vendors_purchase_details_by_description = purchases.groupby(['VendorName', 'Description']).agg(Total_Purchase_price = ('PurchasePrice','sum'),Total_quantity = ('Quantity', 'sum'), Total_Dollars=('Dollars','sum')).reset_index()

In [82]:
vendors_purchase_details_by_description

,VendorName,Description,Total_Purchase_price,Total_quantity,Total_Dollars
0,AAPER ALCOHOL & CHEMICAL CO,Ethyl Alcohol 200 Proof,105.07,1,105.07
1,ADAMBA IMPORTS INTL INC,Bak's Krupnik Honey Liqueur,45.76,39,446.16
2,ADAMBA IMPORTS INTL INC,Vesica Potato Vodka,10103.67,4687,76257.49
3,ADAMBA IMPORTS INTL INC,Vesica Vodka,11.10,6,66.60
4,ALISA CARR BEVERAGES,Ch Cantenac Brown Margaux 11,40.26,8,322.08
...,...,...,...,...,...
9672,ZORVINO VINEYARDS,Zorvino Fragole Z Strawberry,1436.67,1787,16779.93
9673,ZORVINO VINEYARDS,Zorvino Vyds Mango Magnifico,1008.90,1476,12619.80
9674,ZORVINO VINEYARDS,Zorvino Vyds Peachez,946.66,1778,11770.36
9675,ZORVINO VINEYARDS,Zorvino Vyds Pearz,1221.75,2101,14181.75


In [89]:
brands_purchase_details = purchases.groupby(['Brand']).agg(Total_Purchase_price = ('PurchasePrice','sum'),Total_quantity = ('Quantity', 'sum'), Total_Dollars=('Dollars','sum')).reset_index()

In [90]:
brands_purchase_details

,Brand,Total_Purchase_price,Total_quantity,Total_Dollars
0,3140,7027.32,4640,51921.60
1,5215,6681.10,4923,46325.43
2,5255,7984.90,6215,58110.25


In [104]:
vendors_purchase_details_by_description = purchases.groupby(['VendorName','Description']).agg(Total_Purchase_price = ('PurchasePrice','sum'),Total_quantity = ('Quantity', 'sum'), Total_Dollars=('Dollars','sum')).reset_index()

In [105]:
vendors_purchase_details_by_description

,VendorName,Description,Total_Purchase_price,Total_quantity,Total_Dollars
0,AMERICAN VINTAGE BEVERAGE,TGI Fridays Long Island Iced,6681.10,4923,46325.43
1,AMERICAN VINTAGE BEVERAGE,TGI Fridays Orange Dream,7027.32,4640,51921.60
2,AMERICAN VINTAGE BEVERAGE,TGI Fridays Ultimte Mudslide,7984.90,6215,58110.25


In [ ]:
sales = pd.read_sql_query("SELECT * FROM sales", conn)

In [ ]:
sales